In [1]:
#!pip install twython

In [2]:
import re
import string
import csv
import nltk
import unicodedata

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *
from nltk.metrics import ConfusionMatrix
from sklearn.model_selection import train_test_split

import sqlite3
from sqlite3 import Error

In [3]:
def processSentence(sentence):
    # process the tweets

    #Convert to lower case
    sentence = sentence.lower()
    #Convert www.* or https?://* to URL
    sentence = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',sentence)
    #Convert @username to AT_USER
    sentence = re.sub('@[^\s]+','AT_USER',sentence)
    #Remove additional white spaces
    sentence = re.sub('[\s]+', ' ', sentence)
    #Replace #word with word
    sentence = re.sub(r'#([^\s]+)', r'\1', sentence)
    #trimd
    sentence = sentence.strip('\'"')
    #retira açentos e caracteres especiais
    sentence = str(unicodedata.normalize('NFKD', sentence).encode('ascii','ignore'),'utf8')
    #************************* 
    sentence = sentence.replace("tmb", "tambem")
    sentence = sentence.replace(" ta ", " esta ")
    sentence = sentence.replace("tbm", "tambem")
    sentence = sentence.replace("vc", "voce")
    sentence = sentence.replace("sdd", "saudade")
    sentence = sentence.replace("sdds", "saudade")
    sentence = sentence.replace("dnv", "novamente")
    sentence = sentence.replace("pfvr", "por favor")
    sentence = sentence.replace("/", " ")
    sentence = sentence.replace("numca", "nunca")
    sentence = sentence.replace("magico", "magica")
    sentence = sentence.replace("lindo", "linda")
    sentence = sentence.replace("amore", "amor")
    sentence = sentence.replace(" n ", " nao ")
    sentence = sentence.replace("ambas", "ambos")
    sentence = sentence.replace("tjm", "estamos juntos")
    return sentence

In [4]:
#initialize stopWords
stopWords = []

#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    #return pattern.sub(r"\1\1", s)
    return pattern.sub(r"\1", s)
#end

#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    
    if 1==1:
        stopWords = nltk.corpus.stopwords.words('portuguese')
    else:
        fp = open(stopWordListFileName, 'r')
        line = fp.readline()
        while line:
            word = line.strip()
            stopWords.append(word)
            line = fp.readline()
        fp.close()
        
    #****************************************
    stopWords.append('AT_USER')
    stopWords.append('URL')
    stopWords.append('url')
    stopWords.append('pra')
    stopWords.append('q')
    stopWords.append('at_user')
    stopWords.append('poder')
    stopWords.append('profunda')
    stopWords.append('voce')
    stopWords.append('to')    
    stopWords.append('tao') 
    #*****************************************
    stopWords.remove('nem')
    #stopWords.remove('cabeca')
    #stopWords.remove('cabeça')
    #*****************************************
            
    return stopWords
#end

In [5]:
#start getfeatureVector
def getFeatureVector(sentence,stopWords):
    featureVector = []
    #split tweet into words
    words = sentence.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.!')
        if len(w)>1:
            if 1!=1:
                #print(len(w))
                aux = w
                stemmer = nltk.stem.SnowballStemmer('portuguese')
                w = stemmer.stem(w)
                #if w=='indo':
                    #print(aux)
            #check if the word stats with an alphabet
            val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
            #ignore if it is a stop word
            if(w in stopWords or val is None):
                continue
            else:
                #processa plural
                w = processa_plural(w)
                featureVector.append(w.lower())

    return featureVector
#end


In [6]:
def processa_plural(word):
    dict_plural ={'marques':'','rs':'','atras':'','aspas':'','apos':'','antes':'','alas':'','ademais':'','quantas':'','vamos':'','eses':'','esses':'','vezes':'','damos':'','queremos':'','apenas':'','duas':'','dois':'','atraves':'','caos':'','demais':'','ambos':'','detras':''}
    if dict_plural.get(word)==None:
        if re.search(r's$', word):

            if re.search(r'as$', word):
                word = re.sub(r'irmas$', 'irma', word)
                word = re.sub(r'ais$', 'al', word)
                word = re.sub(r'aos$', 'ao', word)
                word = re.sub(r'as$', 'a', word)
                
            elif re.search(r'es$', word):
                if re.search(r'maes$', word):
                    word = 'maes'
                elif re.search(r'bres$', word):
                    word = re.sub(r'bres$', 'bre', word)
                elif re.search(r'zes$', word):
                    word = re.sub(r'zes$', 'z', word)
                elif re.search(r'tres$', word):
                    word = re.sub(r'tres$', 'tre', word)
                elif re.search(r'ores$', word):
                    word = re.sub(r'tres$', 'tre', word)
                else:
                    word = re.sub(r'oes$', 'ao', word)
                    word = re.sub(r'aes$', 'ao', word)
                    word = re.sub(r'gues$','gues', word)
                    word = re.sub(r'res$', 'r', word)
                    word = re.sub(r'es$', 'e', word)

            elif re.search(r'is$', word):
                if re.search(r'veis$', word):
                    word = re.sub(r'veis$', 'vel', word)
                else:
                    word = re.sub(r'ais$', 'al', word)
                    word = re.sub(r'zis$', 'zil', word)
                    word = re.sub(r'eis$', 'il', word)

            elif re.search(r'os$', word):
                if re.search(r'emos$', word):
                    word = re.sub(r'emos$', 'emos', word)
                elif re.search(r'amos$', word):
                    word = re.sub(r'amos$', 'amos', word)
                elif re.search(r'armos$', word):
                    word = re.sub(r'armos$', 'armos', word)
                else:
                    word = re.sub(r'ois$', 'oi', word)
                    word = re.sub(r'os$', 'o', word)

            elif re.search(r'us$', word):
                word = re.sub(r'eus$', 'eu', word)
                word = re.sub(r'us$', 'u', word)

            else:
                word = re.sub(r'ns$', 'm', word)

    return word
    

In [7]:
def extract_features(sentence):
    sentence_words = set(sentence)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in sentence_words)
    return features
#end

In [8]:
def features(sentence):
    words = sentence.lower().split()
    d={}
    excessoes = ['nao','tambem']
    if len(words)>1:
        for w in words:
            if len(w)>1:
                d['contains(%s)' % w]=True
    elif len(words)==1:
        if (words[0] in excessoes)==False:
            d['contains(%s)' % words[0]]=True

    return d

In [1]:
def carrega_base_treino():
    conn = sqlite3.connect('tweets.sqlite')
    cur = conn.cursor()
    #cur.execute("SELECT * FROM tweet where tweet_id = '0' and text like '%não%inútil%'")
    cur.execute("SELECT reacao,sentimento FROM reacao where mark_for_train=1")
    base_treino = cur.fetchall()

    for row in base_treino:
        #print(row)
        cur.execute("insert into tweet (tweet_id, text, sentiment) values ('0', '" + row[0] + "', '" + row[1] + "')").fetchone()
        conn.commit()
    
    conn.close()

In [9]:
#Read the sentence one by one and process it
#inpTweets = csv.reader(open('BaseReLi2.csv', 'r'), delimiter=',')

conn = sqlite3.connect('tweets.sqlite')
cur = conn.cursor()
cur.execute("SELECT * FROM tweet where tweet_id == '0'")
inpSentence = cur.fetchall()
    
stopWords = getStopWordList('portuguese')
featureList = []
featureVectorPositive=[]
featureVectorNonPositive=[]

# Get tweet words
cont_neg=0
cont_pos=0
cont_neu=0
cont_max=3000
item_vec_max = 1

for row in inpSentence:
    
    if len(row)==4:
        if row[3] == 1 and cont_pos <= cont_max:
            cont_pos +=1
            sentiment = row[3]
            sentence = row[2]
            processedSentence = processSentence(sentence)
            featureVector = getFeatureVector(processedSentence, stopWords)
            if len(featureVector)>item_vec_max:
                #tupla = list(nltk.bigrams(featureVector))
                #for t in tupla:
                    #featureVector.append("_".join(" ".join(t).split()))
                featureVectorPositive.append(" ".join(featureVector))
                
                1==1
            else:
                #print(featureVector, sentiment)
                1==1

        elif row[3] == -1 and cont_neg <= cont_max:
            cont_neg +=1
            sentiment = row[3]
            sentence = row[2]
            processedSentence = processSentence(sentence)
            featureVector = getFeatureVector(processedSentence, stopWords)
            if len(featureVector)>item_vec_max:
                #tupla = list(nltk.bigrams(featureVector))
                #for t in tupla:
                    #featureVector.append("_".join(" ".join(t).split()))                    
                featureVectorNonPositive.append(" ".join(featureVector))
                
                1==1
            else:
                #print(featureVector, sentiment)
                1==1

        elif row[3] == 0 and cont_neu <= cont_max:
            cont_neu +=1
            sentiment = row[3]
            sentence = row[2]
            processedSentence = processSentence(sentence)
            featureVector = getFeatureVector(processedSentence, stopWords)
            if len(featureVector)>item_vec_max:
                #tupla = list(nltk.bigrams(featureVector))
                #for t in tupla:
                    #featureVector.append("_".join(" ".join(t).split()))                    
                featureVectorPositive.append(" ".join(featureVector))
                
                1==1
            else:
                #print(featureVector, sentiment)
                1==1
conn.close()
#end loop

In [10]:
featureVectorNonPositive = list(map(features, featureVectorNonPositive))

In [11]:
featureVectorPositive = list(map(features, featureVectorPositive))

In [12]:
FVP_train ,FVP_test = train_test_split(featureVectorPositive,random_state=24061976,shuffle=True,test_size=0.10)
FVnP_train ,FVnP_test = train_test_split(featureVectorNonPositive,random_state=24061976,shuffle=True,test_size=0.10)

NAIVE BAYES CLASSIFIER

In [13]:
from nltk.classify import PositiveNaiveBayesClassifier
classifier = PositiveNaiveBayesClassifier.train(featureVectorPositive,featureVectorNonPositive)

In [14]:
carrega_base_treino()
minDicLen = 0
test = 'Jordana Marques'
processedTestTweet = processSentence(test)
print(processedTestTweet)
test = " ".join((getFeatureVector(processedTestTweet,stopWords)))
#tupla = list(nltk.bigrams(test.split()))

#for t in tupla:
    #test = test + " " + "_".join(" ".join(t).split())
        
print(test)
print(features(test))
dic = features(test)
if len(dic)>minDicLen:
    print(classifier.classify(dic))
else:
    print(None)

jordana marques
jordana marques
{'contains(jordana)': True, 'contains(marques)': True}
True


In [15]:
classifier.prob_classify(dic).prob(False)


0.5

In [ ]:
print(classifier.show_most_informative_features(10000))

In [ ]:
test_result = []
gold_result = []

for i in range(0,len(FVnP_test)):
    #test_result.append(classifier.classify(FVP_test[i])
    test_result.append(str(classifier.classify(FVnP_test[i])))
    gold_result.append('False')
    if test_result != gold_result:
        dict = {}
        dict = FVnP_test[i]
        list_key = list(dict.keys())
        #print("Classificação: " + 'True')
        #print("Palavras".center(70,"*"))
        for c in range(0,len(list_key)):
            if dict.get(list_key[c])==True:
                #print(list_key[c]) 
                1==1
                
        #print("Fim".center(70,"*"))

CM = nltk.ConfusionMatrix(gold_result, test_result)
print(CM)    

In [ ]:
labels = {'True', 'False'}

from collections import Counter
TP, FN, FP = Counter(), Counter(), Counter()
for i in labels:
    for j in labels:
        if i == j:
            TP[i] += int(CM[i,j])
        else:
            FN[i] += int(CM[i,j])
            FP[j] += int(CM[i,j])

print("label\tprecision\trecall\tf_measure")
for label in sorted(labels):
    precision, recall = 0, 0
    if TP[label] == 0:
        f_measure = 0
    else:
        precision = float(TP[label]) / (TP[label]+FP[label])
        recall = float(TP[label]) / (TP[label]+FN[label])
        f_measure = float(2) * (precision * recall) / (precision + recall)
    print(label+"\t"+str(precision)+"\t"+str(recall)+"\t"+str(f_measure))
    